In [2]:
import search
reload(search)

# reopen index, but reload it instead of creating it afresh
udacity_se = search.UdacitySearchEngine(create=False)
    
# dictionary of results
results = udacity_se.search("apple")
print results

[{'slug': u'grand-central-dispatch-gcd--ud576', 'title': u'Grand Central Dispatch (GCD)'}, {'slug': u'intro-to-ios-app-development-with-swift--ud585', 'title': u'Intro to iOS App Development with Swift'}, {'slug': u'swift-for-beginners--ud1022', 'title': u'Swift for Beginners'}, {'slug': u'server-side-swift--ud1031', 'title': u'Server-Side Swift'}, {'slug': u'learn-swift-programming-syntax--ud902', 'title': u'Learn Swift Programming Syntax'}, {'slug': u'ios-persistence-and-core-data--ud325', 'title': u'iOS Persistence and Core Data'}]


In [7]:
import search
reload(search)

# reopen index, but reload it instead of creating it afresh
se = search.HarvardXSearchEngine(create=False)
    
# dictionary of results
results = se.search("Psychology")
print results

OSError: [Errno 2] No such file or directory: 'search_indices/whoosh_indices/harvardx'

In [ ]:
# compare query rewriting to the normal results 
import rewriter
reload(rewriter)

import search
reload(search)

searches = [
    "neural networks",
    "swift",
    "venture capital",
    "object-oriented programming",
    "deep learning",
    "macOS",
    "einstein"
]


udacity_se = search.UdacitySearchEngine(create=False)
rw = rewriter.WikipediaRewriter()
cw = rewriter.ControlRewriter()

wikidacity = rewriter.RewritingSearchEngine(rw, udacity_se)
plaindacity = rewriter.RewritingSearchEngine(cw, udacity_se)

for query in searches:
    # normal results
    num_plain_results = len(plaindacity.search(query))
    
    # with wikipedia rewriting
    num_wiki_results = len(wikidacity.search(query))
    
    print (query, num_plain_results, num_wiki_results)

In [6]:
import rewriter
reload(rewriter)

wv = rewriter.Word2VecRewriter(create=True)

IOError: [Errno 2] No such file or directory: 'models/whoosh_indices/word2vec/word2vec.wv.syn0.npy'

In [ ]:
worddacity = rewriter.RewritingSearchEngine(wv, udacity_se)

In [ ]:
for query in searches:
    # normal results
    num_plain_results = len(plaindacity.search(query))

    num_word_results = len(worddacity.search(query))
    
    print (query, num_plain_results, num_word_results)

In [87]:
# compare query rewriting to the normal results 
import rewriter
reload(rewriter)

import search
reload(search)

hx_se = search.HarvardXSearchEngine(create=False)

cw = rewriter.ControlRewriter()
rw = rewriter.WikipediaRewriter()
wv = rewriter.Word2VecRewriter(create=False)

rewrite_engines = [
    rewriter.RewritingSearchEngine(cw, hx_se),
    rewriter.RewritingSearchEngine(rw, hx_se),
    rewriter.RewritingSearchEngine(wv, hx_se)
]

In [77]:
# TODO: word2vec doesn't work on multiple-word queries. we need to add ngram support
searches = [
    "confirmation bias",
    "biochemistry",
    "deep learning",
    "existentialism"
]



for query in searches:
    result_nums = [len(qrse.search(query)) for qrse in rewrite_engines]
    
    print (query, result_nums)

('confirmation bias', [2, 49, 2])
('biochemistry', [10, 37, 101])
('deep learning', [10, 13, 10])
('existentialism', [0, 20, 53])


In [90]:
wv.rewrite("biochemistry")

[u'high-temperature',
 u'Games',
 u'cryptography',
 u'research',
 u'organisms',
 u'multiverse',
 u'organizational',
 u'models',
 u'bias',
 u'qualitative',
 u'biochemistry']

In [92]:

import search
reload(search)

edx = search.EdXSearchEngine(create=True)

In [94]:
edx.search("civil war")

[{'course_id': u'ColumbiaX/HIST1.1x/3T2014',
  'name': u'Civil War and Reconstruction - 1850-1861'},
 {'course_id': u'ColumbiaX/HIST1.2x/3T2014',
  'name': u'Civil War and Reconstruction - 1861-1865'},
 {'course_id': u'ColumbiaX/HIST1.3x/1T2015',
  'name': u'The Civil War and Reconstruction - 1865-1890'},
 {'course_id': u'course-v1:ColumbiaX+HIST1.1x+3T2015',
  'name': u'The Civil War and Reconstruction - 1850-1861'},
 {'course_id': u'course-v1:ColumbiaX+HIST1.2x+3T2015',
  'name': u'The Civil War and Reconstruction - 1861-1865'},
 {'course_id': u'course-v1:ColumbiaX+HIST1.3x+1T2016',
  'name': u'The Civil War and Reconstruction - 1865-1890'},
 {'course_id': u'course-v1:HarvardX+AmPoX.5+2T2015',
  'name': u'Poetry in America: The Civil War and Its Aftermath'}]

In [95]:
hx_se.search("ed")

[{'course_id': u'course-v1:HarvardX+MCB63X+3T2015',
  'display_name': u'Question 3'},
 {'course_id': u'course-v1:HarvardX+MCB63X+3T2015',
  'display_name': u'Question 2'},
 {'course_id': u'course-v1:HarvardX+GSE2x+1T2016',
  'display_name': u'Image Credits'},
 {'course_id': u'course-v1:HarvardX+HUM1.3x+3T2015',
  'display_name': u'Reference'},
 {'course_id': u'course-v1:HarvardX+MUS24.2x+1T2016',
  'display_name': u'Credits'},
 {'course_id': u'course-v1:HarvardX+MCB63X+3T2015',
  'display_name': u'Glucose Catabolism in Bacteria and Fermentation'},
 {'course_id': u'course-v1:HarvardX+MCB63X+3T2015',
  'display_name': u'Question 1'},
 {'course_id': u'course-v1:HarvardX+MUS24.1x+2T2016',
  'display_name': u'Credits'},
 {'course_id': u'course-v1:HarvardX+MUS24.1x+2T2016',
  'display_name': u'Tu se\u2019 morta: Musical Expression '},
 {'course_id': u'course-v1:HarvardX+HDS3221.4x+2T2016',
  'display_name': u'Resources for Further Study'}]

In [96]:
cw = rewriter.ControlRewriter()
rw = rewriter.WikipediaRewriter()
wv = rewriter.Word2VecRewriter(create=False)

rewrite_engines = [
    rewriter.RewritingSearchEngine(cw, edx),
    rewriter.RewritingSearchEngine(rw, edx),
    rewriter.RewritingSearchEngine(wv, edx)
]

In [97]:
# TODO: word2vec doesn't work on multiple-word queries. we need to add ngram support
searches = [
    "confirmation bias",
    "biochemistry",
    "deep learning",
    "existentialism"
]



for query in searches:
    result_nums = [len(qrse.search(query)) for qrse in rewrite_engines]
    
    print (query, result_nums)

('confirmation bias', [0, 4, 0])
('biochemistry', [1, 10, 23])
('deep learning', [0, 0, 0])
('existentialism', [0, 0, 12])


In [1]:
### FORMAL DEMOS

In [12]:
## SEARCH ENGINE DEMOS

import search
from pprint import pprint

# Create a search engine that searches over all edX courses.
# Under the hood, this uses Python's Whoosh library to index
# the course data stored in a CSV and then run searches against it.
edx_engine = search.EdXSearchEngine(create=False)

# We expose a simple searching API
edx_results = edx_engine.search("biology")
pprint(edx_results)

# You could also search over HarvardX resources or Udacity courses
# with similar classes!

[{'course_id': u'MITx/7.28.1x/1T2015', 'name': u'Molecular Biology'},
 {'course_id': u'course-v1:RICEx+AdvBIOx+3T2016',
  'name': u'AP\xc2\xae Biology'},
 {'course_id': u'course-v1:HarvardX+MCB64.1x+2T2016',
  'name': u'Cell Biology: Mitochondria'},
 {'course_id': u'course-v1:IEEEx+SysBio1x+2016_T1',
  'name': u'Introduction to Systems Biology.'},
 {'course_id': u'course-v1:IEEEx+SysBio1x+3T2016',
  'name': u'Introduction to Systems Biology.'},
 {'course_id': u'MITx/7.QBWx/2T2014',
  'name': u'Quantitative Biology Workshop'},
 {'course_id': u'MITx/7.QBW_1x/1T2015',
  'name': u'Quantitative Biology Workshop'},
 {'course_id': u'course-v1:MITx+20.305x+3T2015',
  'name': u'Principles of Synthetic Biology'},
 {'course_id': u'course-v1:MITx+7.QBWx_3+1T2016',
  'name': u'Quantitative Biology Workshop'},
 {'course_id': u'course-v1:MITx+7.QBWx_4+3T2016',
  'name': u'Quantitative Biology Workshop'}]


In [26]:
## QUERY REWRITING DEMOS

import rewriter

# Query rewriting lets you turn a single search query into
# multiple related queries. You can then search for *all*
# of these queries, which can result in more and more useful
# results than just the original query would give.

# First, a rewriter that uses the Wikipedia category API
# to find terms related to the original term
wiki_rewriter = rewriter.WikipediaRewriter()
wiki_rewritten_queries = wiki_rewriter.rewrite("newton's laws of motion")
pprint(wiki_rewritten_queries)


# Second, a rewriter that uses Word2Vec to find similar
# words to the entered term. This is a machine learning
# algorithm trained on a large text corpus (incidentally, a dump
# of all Wikipedia articles.)
w2v_rewriter = rewriter.Word2VecRewriter(create=False)
w2v_rewritten_queries = w2v_rewriter.rewrite("socialism")
pprint(w2v_rewritten_queries)

['All articles with unsourced statements',
 'Articles containing video clips',
 'Articles with unsourced statements from August 2012',
 'Classical mechanics',
 'Commons category with local link different than on Wikidata',
 'Concepts in physics',
 'Copernican Revolution',
 'EngvarB from July 2014',
 'Experimental physics',
 'History of physics',
 "newton's laws of motion"]
[u'injunctions',
 u'governance,',
 u'libertarians',
 u'social,',
 u'justice,',
 u'self-determination,',
 u'liberalism,',
 u'disregard',
 u'neutrality',
 u'ideological',
 u'socialism']


In [31]:
## PUTTING IT ALL TOGETHER

# We can apply a query rewriter to a search engine
# to create a new search engine that runs the
# query rewriter on all incoming searches and returns
# all results it gets from the queries

# For example, imagine trying the Word2Vec rewriter
# on the edX search engine 
w2v_edx_engine = rewriter.RewritingSearchEngine(w2v_rewriter, edx_engine)
rewritten_results = w2v_edx_engine.search("calculus")
pprint(rewritten_results)

[{'course_id': u'SchoolYourself/GeometryX/1T2015',
  'name': u'Introduction to Geometry'},
 {'course_id': u'course-v1:SchoolYourself+GeometryX+2T2016',
  'name': u'Introduction to Geometry'},
 {'course_id': u'course-v1:TsinghuaX+70240183x+3T2015',
  'name': u'Computational Geometry'},
 {'course_id': u'course-v1:TsinghuaX+70240183x+3T2016',
  'name': u'Computational Geometry'},
 {'course_id': u'course-v1:TeachForAmericaX+HSMATH2.1x+3T2016',
  'name': u'How to Teach High School Geometry'},
 {'course_id': u'course-v1:GeorgetownX+PHYX-008-01x+1T2017',
  'name': u'Quantum Mechanics'},
 {'course_id': u'MITx/8.01x/2013_SOND', 'name': u'Classical Mechanics'},
 {'course_id': u'MITx/8.MReVx/2T2014', 'name': u'Mechanics ReView'},
 {'course_id': u'course-v1:UrFUx+ENGM1.1.x+3T2016',
  'name': u'Engineering Mechanics'},
 {'course_id': u'course-v1:MITx+8.05.1x+1T2016',
  'name': u'Mastering Quantum Mechanics Part I: Wave Mechanics'},
 {'course_id': u'MITx/8.05x/1T2015', 'name': u'Mastering Quantum Me